# [Final Project Demo] 딥리서치 에이전트

### 실습 목표

1. `LangGraph`, `CrewAI`, `ADK`의 개념을 하나의 워크플로우 안에서 엮습니다.
2. `FastAPI` 기반의 MCP(자원 허브) 서버와 A2A(Agent-to-Agent) 게이트웨이를 통해, 확장 가능하고 유지보수가 용이한 Agent 아키텍처를 설계하고 운영합니다.
3. `Redis`와 `ChromaDB`를 활용한 `MemorySystem`을, 과거의 경험과 사용자 피드백으로부터 학습하고 진화하는 Agent를 구현합니다.
4. `Langfuse`를 시스템 전체에 통합하여, 워크플로우를 추적하고, `Langfuse Dataset`과 `LLM-as-a-Judge`를 통해 시스템의 성능을 정량적으로 평가하고 개선하는 LLMOps/EvalOps 사이클을 수행합니다.

### 0. 준비

1일부터 5일까지 사용했던 모든 라이브러리를 설치하고, Langfuse를 포함한 모든 외부 서비스의 API 키를 설정합니다.

In [ ]:
# !pip install fastapi uvicorn python-dotenv nest-asyncio aiohttp httpx newsapi-python arxiv tavily-python slowapi langfuse crewai crewai-tools google-adk langchain-google-genai pydantic instructor requests jsonref starlette redis chromadb sentence-transformers wolframalpha langgraph langchain -q

In [ ]:
import os

# LLM 및 기본 Tool 키
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY"

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = "YOUR_API_KEY"

if "NEWS_API_KEY" not in os.environ:
    os.environ["NEWS_API_KEY"] = "YOUR_API_KEY"

if "WOLFRAM_ALPHA_APP_ID" not in os.environ:
    os.environ["WOLFRAM_ALPHA_APP_ID"] = "YOUR_API_KEY"

# Langfuse Observability/Evaluation Platform 키
if "LANGFUSE_PUBLIC_KEY" not in os.environ:
    os.environ["LANGFUSE_PUBLIC_KEY"] = "YOUR_API_KEY"

if "LANGFUSE_SECRET_KEY" not in os.environ:
    os.environ["LANGFUSE_SECRET_KEY"] = "YOUR_API_KEY"

os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"

# 서버 간 통신을 위한 마스터 인증 키
os.environ["MASTER_API_KEY"] = "samsung-llm-agent-lv4-master-key"

# CrewAI 호환성을 위한 GEMINI_API_KEY 설정
if "GEMINI_API_KEY" not in os.environ:
    os.environ["GEMINI_API_KEY"] = os.environ["GOOGLE_API_KEY"]

# --- 외부 서비스 연결 정보 설정 ---
if "UPSTASH_REDIS_REST_URL" not in os.environ:
    os.environ["UPSTASH_REDIS_REST_URL"] = "YOUR_API_KEY"
if "UPSTASH_REDIS_REST_TOKEN" not in os.environ:
    os.environ["UPSTASH_REDIS_REST_TOKEN"] = "YOUR_API_KEY"

### 1. 스크립트 생성

앞선 5일간 설계하고 구현했던 모든 구성 요소들을 독립적인 Python 파일로 다시 생성합니다.
  
가장 먼저, 모든 Agent들의 MCP 서버와 그 능력이 될 Tool들을 구현합니다.

#### 1.1 Tool 구현 (`server_tools.py`)

기존의 검색 Tool (`web_search`, `news_api_search`, `arxiv_search`)에 더해, 이번 최종 프로젝트의 복잡한 주제를 해결하기 위한 3개의 새로운 전문가 Tool을 추가합니다.

- `python_calculator`: 시장 성장률 예측이나 PPD(Pixels Per Degree) 같은 정량적 데이터를 계산하기 위한 Tool입니다.
- `ethics_check`: Embodied AI의 프라이버시 문제와 같은 민감한 주제를 다룰 때, 글의 내용이 윤리적으로 편향되지 않았는지 검토하는 LLM 기반 평가 Tool입니다.
- `mermaid_generator`: 복잡한 기술 스택이나 시스템 아키텍처를 독자가 쉽게 이해할 수 있도록, 텍스트 설명을 바탕으로 Mermaid 다이어그램 코드를 생성하는 LLM 기반 시각화 Tool입니다.

In [ ]:
%%writefile server_tools.py

import os
import json
from tavily import TavilyClient
from newsapi import NewsApiClient
import arxiv
from langchain_google_genai import ChatGoogleGenerativeAI

# --- Tool 내부에서 사용할 LLM 초기화 ---
tool_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0.0)

# --- 1. 기존 검색 Tool들 ---
async def web_search(query: str) -> str:
    """웹에서 최신 정보를 검색합니다."""
    try:
        client = TavilyClient(api_key=os.environ['TAVILY_API_KEY'])
        response = client.search(query=query, max_results=5, search_depth="advanced")
        results = [{'title': obj['title'], 'url': obj['url'], 'content': obj['content'][:500]} for obj in response['results']]
        return json.dumps(results, ensure_ascii=False)
    except Exception as e:
        return json.dumps({"error": f"Tavily API 오류: {e}"}, ensure_ascii=False)

async def news_api_search(query: str) -> str:
    """최신 뉴스 기사를 검색합니다."""
    try:
        client = NewsApiClient(api_key=os.environ['NEWS_API_KEY'])
        response = client.get_everything(q=query, language='ko', sort_by='relevancy', page_size=3)
        if response['status'] == 'ok':
            articles = [{'title': article['title'], 'url': article['url'], 'description': article['description']} for article in response['articles']]
            return json.dumps(articles, ensure_ascii=False)
        else:
            return json.dumps({"error": f"News API 오류: {response.get('message', 'Unknown error')}"}, ensure_ascii=False)
    except Exception as e:
        return json.dumps({"error": f"News API 클라이언트 오류: {e}"}, ensure_ascii=False)

async def arxiv_search(query: str) -> str:
    """학술 논문을 Arxiv에서 검색합니다."""
    try:
        client = arxiv.Client()
        search = arxiv.Search(query=query, max_results=2, sort_by=arxiv.SortCriterion.Relevance)
        results = list(client.results(search))
        papers = [{'title': result.title, 'authors': [str(a) for a in result.authors], 'summary': result.summary, 'pdf_url': result.pdf_url} for result in results]
        return json.dumps(papers, ensure_ascii=False)
    except Exception as e:
        return json.dumps({"error": f"Arxiv 검색 오류: {e}"}, ensure_ascii=False)

# --- 2. 신규 Tool들 ---

async def python_calculator(code: str) -> str:
    """주어진 Python 코드를 실행하여 수학적/공학적 계산을 수행합니다. 간단한 계산만 가능합니다."""

    try:
        result = eval(code, {"__builtins__": {}}, {})
        return json.dumps({"result": str(result)})
    except Exception as e:
        return json.dumps({"error": f"코드 실행 오류: {e}"}, ensure_ascii=False)

async def ethics_check(text: str) -> str:
    """주어진 텍스트에 잠재적인 윤리적 문제가 있는지(편향, 프라이버시 침해 등) 검토합니다."""
    try:
        prompt = f"""당신은 AI 윤리 전문가입니다. 다음 텍스트를 분석하여 잠재적인 윤리적 문제(예: 편향, 개인정보 침해, 사회적 갈등 조장 등)가 있는지 검토하고, '문제 없음' 또는 '주의 필요' 판정과 함께 그 근거를 간결하게 설명해주세요.\n\n[검토할 텍스트]:\n{text}"""
        response = tool_llm.invoke(prompt).content
        return json.dumps({"ethics_review": response}, ensure_ascii=False)
    except Exception as e:
        return json.dumps({"error": f"윤리 검토 중 오류 발생: {e}"}, ensure_ascii=False)

async def mermaid_generator(description: str) -> str:
    """텍스트 설명을 바탕으로 Mermaid.js 다이어그램 코드를 생성합니다."""
    try:
        prompt = f"""당신은 Mermaid.js 다이어그램 전문가입니다. 다음 설명을 보고, 이를 시각화하는 Mermaid 코드 블록(```mermaid ... ```)을 생성해주세요. 다른 설명은 붙이지 말고 오직 코드 블록만 생성해야 합니다.\n\n[설명]:\n{description}"""
        response = tool_llm.invoke(prompt).content
        # LLM 응답에서 코드 블록만 정확히 추출
        import re
        code_match = re.search(r"```mermaid\n(.*?)```", response, re.DOTALL)
        if code_match:
            return json.dumps({"mermaid_code": code_match.group(1).strip()}, ensure_ascii=False)
    except Exception as e:
        return json.dumps({"error": f"Mermaid 코드 생성 중 오류 발생: {e}"}, ensure_ascii=False)

#### 1.2 MCP 서버의 지식: 정적 리소스 구현 (`server_resources.py`)

Agent들이 공통적으로 참고해야 할 정적인 데이터(블로그 템플릿, 스타일 가이드 등)를 제공하는 리소스 모듈을 생성합니다.

In [ ]:
%%writefile server_resources.py

BLOG_TEMPLATES = {
    "deep_dive_tech_analysis": "## 1. 서론: 기술의 등장 배경 및 중요성\n\n## 2. 핵심 아키텍처 및 작동 원리\n   - ### 2.1. 하드웨어 구성 요소\n   - ### 2.2. 소프트웨어 스택\n\n## 3. 주요 기술적 과제 및 해결 방안\n\n## 4. 산업별 파급 효과 및 시장 전망\n\n## 5. 사회/윤리적 고찰\n\n## 6. 결론: 미래 전망 및 제언",
    "market_trend_report": "## 1. 시장 개요 및 핵심 트렌드\n\n## 2. 주요 플레이어 및 전략 분석\n   - ### 2.1. 선두 기업 (Apple, Meta 등)\n   - ### 2.2. 도전 기업 (스타트업 등)\n\n## 3. 정량적 시장 데이터 분석\n\n## 4. 향후 5년간 시장 예측\n\n## 5. 결론 및 투자 인사이트"
}

STYLE_GUIDES = {
    "samsung_technical_blog": "문체: 전문적이면서도 최대한 명확하고 간결하게.\n대상 독자: 기술에 대한 깊은 이해를 가진 개발자 및 연구원.\n어조: 객관적이고, 데이터와 사실에 기반하며, 신뢰감을 주는 톤앤매너를 유지.\n참고: 모든 기술 용어는 첫 등장 시 영문 표기를 병기.",
    "general_public_tech_column": "문체: 친근하고 이해하기 쉽게, 비유와 예시를 적극적으로 사용.\n대상 독자: 기술에 관심은 많지만 전문가는 아닌 일반 대중.\n어조: 흥미를 유발하고, 기술이 우리 삶에 미치는 영향을 중심으로 서술."
}

async def get_template(template_id: str):
    return BLOG_TEMPLATES.get(template_id, {"error": "Template not found"})

async def get_style_guide(guide_id: str):
    return STYLE_GUIDES.get(guide_id, {"error": "Style guide not found"})

#### 1.3 MCP 서버 본체 구현 (`mcp_server.py`)

이제 Tool과 리소스를 API 엔드포인트로 노출하고, 인증(Authentication), 요청 수 제한(Rate Limiting), 관측 가능성(Observability) 기능을 갖춘 MCP 서버의 코드를 생성합니다.

In [ ]:
%%writefile mcp_server.py

from fastapi import FastAPI, HTTPException, Depends, Security, Request
from pydantic import BaseModel, Field
import os
from fastapi.security import APIKeyHeader
from slowapi import Limiter, _rate_limit_exceeded_handler
from slowapi.util import get_remote_address
from slowapi.errors import RateLimitExceeded
from langfuse import observe

# 모듈화된 Tool과 리소스 함수들을 임포트
from server_tools import (
    web_search, news_api_search, arxiv_search,
    python_calculator, ethics_check, mermaid_generator
)
from server_resources import get_template, get_style_guide

# FastAPI 앱 및 안정성/보안 기능 초기화
app = FastAPI(title="MCP Server for Sentient Architect", version="2.0.0")
limiter = Limiter(key_func=get_remote_address)
app.state.limiter = limiter
app.add_exception_handler(RateLimitExceeded, _rate_limit_exceeded_handler)

API_KEY_NAME = "X-API-Key"
api_key_header = APIKeyHeader(name=API_KEY_NAME, auto_error=False)

async def get_api_key(api_key: str = Security(api_key_header)):
    if api_key == os.environ.get("MASTER_API_KEY"):
        return api_key
    else:
        raise HTTPException(status_code=401, detail="Invalid or missing API Key")

# Pydantic 모델 정의
class ToolCallRequest(BaseModel):
    query: str = Field(..., description="Tool에 전달할 검색어 또는 입력값")

class CodeExecutionRequest(BaseModel):
    code: str = Field(..., description="실행할 Python 코드")

class TextAnalysisRequest(BaseModel):
    text: str = Field(..., description="분석할 텍스트")

class DiagramGenerationRequest(BaseModel):
    description: str = Field(..., description="다이어그램으로 변환할 텍스트 설명")

# --- API 엔드포인트 ---

@app.get("/", summary="서버 상태 확인", tags=["Status"])
async def read_root():
    return {"status": "ok", "message": "MCP Server is running and ready to serve."}

# 검색 Tool 엔드포인트
@app.post("/api/v1/tools/web_search", summary="웹 검색", tags=["Search Tools"])
@limiter.limit("30/minute")
@observe(name="mcp-tool-web-search")
async def prod_web_search(request: Request, data: ToolCallRequest, api_key: str = Depends(get_api_key)):
    result = await web_search(data.query)
    return {"tool": "web_search", "query": data.query, "result": result}

@app.post("/api/v1/tools/news_api_search", summary="뉴스 검색", tags=["Search Tools"])
@limiter.limit("30/minute")
@observe(name="mcp-tool-news-api-search")
async def prod_news_search(request: Request, data: ToolCallRequest, api_key: str = Depends(get_api_key)):
    result = await news_api_search(data.query)
    return {"tool": "news_api_search", "query": data.query, "result": result}

@app.post("/api/v1/tools/arxiv_search", summary="논문 검색", tags=["Search Tools"])
@limiter.limit("30/minute")
@observe(name="mcp-tool-arxiv-search")
async def prod_arxiv_search(request: Request, data: ToolCallRequest, api_key: str = Depends(get_api_key)):
    result = await arxiv_search(data.query)
    return {"tool": "arxiv_search", "query": data.query, "result": result}

# 신규 전문가 Tool 엔드포인트
@app.post("/api/v1/tools/python_calculator", summary="Python 코드 계산", tags=["Expert Tools"])
@limiter.limit("60/minute")
@observe(name="mcp-tool-python-calculator")
async def prod_python_calculator(request: Request, data: CodeExecutionRequest, api_key: str = Depends(get_api_key)):
    result = await python_calculator(data.code)
    return {"tool": "python_calculator", "code": data.code, "result": result}

@app.post("/api/v1/tools/ethics_check", summary="윤리성 검토", tags=["Expert Tools"])
@limiter.limit("60/minute")
@observe(name="mcp-tool-ethics-check")
async def prod_ethics_check(request: Request, data: TextAnalysisRequest, api_key: str = Depends(get_api_key)):
    result = await ethics_check(data.text)
    return {"tool": "ethics_check", "text_length": len(data.text), "result": result}

@app.post("/api/v1/tools/mermaid_generator", summary="Mermaid 다이어그램 생성", tags=["Expert Tools"])
@limiter.limit("30/minute")
@observe(name="mcp-tool-mermaid-generator")
async def prod_mermaid_generator(request: Request, data: DiagramGenerationRequest, api_key: str = Depends(get_api_key)):
    result = await mermaid_generator(data.description)
    return {"tool": "mermaid_generator", "description": data.description, "result": result}

# 정적 리소스 엔드포인트
@app.get("/api/v1/resources/templates/{template_id}", summary="템플릿 조회", tags=["Static Resources"])
@limiter.limit("100/minute")
@observe(name="mcp-resource-template")
async def prod_read_template(request: Request, template_id: str, api_key: str = Depends(get_api_key)):
    content = await get_template(template_id)
    if isinstance(content, dict) and "error" in content:
        raise HTTPException(status_code=404, detail=content["error"])
    return {"resource": "template", "id": template_id, "content": content}

@app.get("/api/v1/resources/style_guides/{guide_id}", summary="스타일 가이드 조회", tags=["Static Resources"])
@limiter.limit("100/minute")
@observe(name="mcp-resource-style-guide")
async def prod_read_style_guide(request: Request, guide_id: str, api_key: str = Depends(get_api_key)):
    content = await get_style_guide(guide_id)
    if isinstance(content, dict) and "error" in content:
        raise HTTPException(status_code=404, detail=content["error"])
    return {"resource": "style_guide", "id": guide_id, "content": content}

#### 1.4 멀티 에이전트 통신 프로토콜 구현(`a2a_protocol.py`)

서로 다른 프레임워크로 만들어진 Agent들이 소통하기 위한 표준 데이터 형식을 정의합니다. 이 프로토콜은 시스템의 안정성과 확장성을 보장하는 API 명세(Contract) 역할을 합니다.  

다만 A2A를 실습 환경 상 구현하기 힘들기에, 이와 유사한 FastAPI 기반 호출을 사용합니다.

In [ ]:
%%writefile a2a_protocol.py

from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any

# --- 1. 대화 관리자 (Orchestrator)의 입출력 ---
class OrchestrationRequest(BaseModel):
    session_id: str = Field(description="각 대화 세션을 식별하는 고유 ID")
    user_input: str = Field(description="사용자의 최초 리서치 요청")

class OrchestrationResponse(BaseModel):
    session_id: str
    final_report: str = Field(description="최종적으로 완성된 심층 분석 보고서")
    status: str = Field(default="COMPLETED", description="전체 워크플로우의 최종 상태")
    trace_url: Optional[str] = Field(default=None, description="Langfuse의 전체 실행 과정 추적 URL")
    error_message: Optional[str] = None

# --- 2. 전문가 서비스 간 내부 통신용 모델  ---
class ContentCreationRequest(BaseModel):
    topic: str
    user_preferences: str
    memory_context: str

class QualityControlRequest(BaseModel):
    topic: str
    draft_content: str
    user_preferences: str

#### 1.5 MCP 서버와의 통신 채널: `mcp_client.py`

A2A 게이트웨이 뒤에서 작동하는 전문가 Agent 팀들(특히 CrewAI)이 MCP 서버의 Tool과 리소스를 사용할 수 있도록, API 통신을 캡슐화한 클라이언트 모듈을 생성합니다.

In [ ]:
%%writefile mcp_client.py

import requests
import json

class MCPClient:
    def __init__(self, base_url: str, api_key: str):
        if not base_url:
            raise ValueError("MCP 서버의 base_url이 필요합니다.")
        self.base_url = base_url.rstrip('/')
        self.headers = {
            "X-API-Key": api_key,
            "Content-Type": "application/json",
        }

    def call_tool(self, tool_name: str, payload: dict) -> dict:
        """MCP 서버의 Tool 엔드포인트를 호출합니다. payload는 각 Tool의 Pydantic 모델에 맞춰 전달됩니다."""
        endpoint = f"{self.base_url}/api/v1/tools/{tool_name}"
        try:
            response = requests.post(endpoint, headers=self.headers, json=payload, timeout=180)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            return {"error": f"HTTP 오류: {e.response.status_code}", "detail": e.response.text}
        except requests.exceptions.RequestException as e:
            return {"error": f"Tool 호출 중 네트워크 오류 발생: {e}"}

    def get_resource(self, resource_type: str, resource_id: str) -> dict:
        """MCP 서버의 정적 리소스 엔드포인트를 호출합니다."""
        endpoint = f"{self.base_url}/api/v1/resources/{resource_type}/{resource_id}"
        try:
            response = requests.get(endpoint, headers=self.headers, timeout=30)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            return {"error": f"HTTP 오류: {e.response.status_code}", "detail": e.response.text}
        except requests.exceptions.RequestException as e:
            return {"error": f"리소스 조회 중 네트워크 오류 발생: {e}"}

#### 1.6 전문가 서비스 모듈 구현

이제 각기 다른 프레임워크로 구현된 3개의 핵심 전문가 서비스 로직을 독립된 파일로 생성합니다.  

이 모듈들은 게이트웨이로부터 작업을 위임받아 작업을 수행합니다.

In [ ]:
%%writefile service_crewai_research.py

from crewai import Agent, Task, Crew, Process
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import BaseTool
import json
from mcp_client import MCPClient
from langfuse.langchain import CallbackHandler as LangfuseCallbackHandler
from langfuse import observe

class MCPToolWrapper(BaseTool):
    name: str
    description: str
    client: MCPClient
    payload_keys: list[str]

    @observe(name="crewai-mcp-tool-call")
    def _run(self, kwargs) -> str:
        print(f"  [MCP Bridge] CrewAI -> MCP: Calling tool '{self.name}' with args {kwargs}")
        # Pydantic 모델에 맞는 payload 구성
        payload = {key: kwargs.get(key) for key in self.payload_keys}
        response = self.client.call_tool(self.name, payload)
        return json.dumps(response, ensure_ascii=False)

@observe(name="crewai-research-team-execution")
def run_research_crew(topic: str, user_preferences: str, memory_context: str, mcp_client: MCPClient):
    print(f"[CrewAI Service] 👥 딥 리서치 팀 가동 (주제: {topic[:30]}...)")
    try:
        handler = LangfuseCallbackHandler()
        llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", callbacks=[handler])

        # MCP Tool Wrappers
        web_search = MCPToolWrapper(name="web_search", description="웹 검색", client=mcp_client, payload_keys=["query"])
        arxiv_search = MCPToolWrapper(name="arxiv_search", description="학술 논문 검색", client=mcp_client, payload_keys=["query"])
        news_search = MCPToolWrapper(name="news_api_search", description="최신 뉴스 검색", client=mcp_client, payload_keys=["query"])

        # Agents
        tech_analyst = Agent(role="기술 분석가", goal=f"{topic}의 핵심 기술(HW/SW) 분석", backstory="실리콘밸리 20년 경력의 최고 기술 분석가.", tools=[web_search, arxiv_search], llm=llm, verbose=True)
        market_analyst = Agent(role="시장 분석가", goal="주요 기업 전략 및 시장 동향 분석", backstory="월스트리트 출신 시장 분석 전문가.", tools=[web_search, news_search], llm=llm, verbose=True)
        social_analyst = Agent(role="사회/윤리 분석가", goal="기술의 사회적, 윤리적 영향 분석", backstory="퓨처라마 연구소의 사회학자.", tools=[web_search], llm=llm, verbose=True)

        # Tasks
        tech_task = Task(description=f"'{topic}'의 기술적 측면(센서, 칩셋, SLAM 알고리즘 등)을 분석하라.", expected_output="기술 분석 보고서", agent=tech_analyst)
        market_task = Task(description=f"'{topic}' 관련 시장 동향, 주요 기업(Apple, Meta 등)의 전략을 분석하라.", expected_output="시장 동향 보고서", agent=market_analyst)
        social_task = Task(description=f"'{topic}'이 가져올 사회적, 윤리적 문제(프라이버시, 디지털 격차 등)를 분석하라.", expected_output="사회/윤리 분석 보고서", agent=social_analyst)

        # Final Writer Agent
        report_writer = Agent(role="종합 보고서 작성가", goal="여러 분석 보고서를 종합하여 최종 보고서 초안 작성", backstory="The Economist의 수석 에디터.", llm=llm, verbose=True)
        write_task = Task(description=f"기술, 시장, 사회 분석 보고서를 모두 종합하고, 사용자의 요구사항('{user_preferences}')과 과거의 컨텍스트('{memory_context}')를 반영하여 최종 보고서 초안을 작성하라.", expected_output="완성도 높은 종합 분석 보고서 초안", agent=report_writer, context=[tech_task, market_task, social_task])

        crew = Crew(agents=[tech_analyst, market_analyst, social_analyst, report_writer], tasks=[tech_task, market_task, social_task, write_task], process=Process.hierarchical, manager_llm=llm)
        result = crew.kickoff()
        print("[CrewAI Service]  리서치 및 초안 작성 완료.")
        return result
    except Exception as e:
        return f"CrewAI 실행 중 오류 발생: {e}"

In [ ]:
%%writefile service_adk_quality.py

from langchain_google_genai import ChatGoogleGenerativeAI
from langfuse.langchain import CallbackHandler
from mcp_client import MCPClient
import json
import os

def run_quality_control(topic: str, draft_content: str, mcp_client: MCPClient):
    print(f"[ADK Service] 🧐 품질 관리 및 편집 작업 시작...")
    handler = CallbackHandler()

    try:
        ethics_review_res = mcp_client.call_tool("ethics_check", {"text": draft_content})
        ethics_review = ethics_review_res.get('result', '{"ethics_review": "N/A"}')
    except Exception as e:
        ethics_review = '{"ethics_review": "N/A"}'
        print(f"! ADK Service: ethics_check 도구 호출 실패: {e}")

    try:
        readability_res = mcp_client.get_resource("style_guides", "samsung_technical_blog")
        style_guide = readability_res.get('content', 'N/A')
    except Exception as e:
        style_guide = 'N/A'
        print(f"! ADK Service: style_guides 리소스 조회 실패: {e}")

    editor_llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.1, callbacks=[handler])


    prompt = f"""당신은 삼성전자 기술 블로그의 최고 수준 편집자입니다. 다음 정보를 바탕으로 주어진 초안을 최종 발행본으로 완성해주세요.

    [주제]: {topic}
    [초안]:
{draft_content}
    [스타일 가이드]:
{style_guide}
    [윤리성 검토 결과]:
{ethics_review}

    [요청사항]:
    1. 스타일 가이드를 완벽하게 준수하여 문체를 수정하세요.
    2. 윤리성 검토 결과를 반영하여 문제가 될 수 있는 표현을 수정하세요.
    3. 전체적인 논리 흐름을 강화하고, 독자가 이해하기 쉽도록 문장을 다듬어주세요.
    4. 최종적으로, 독자에게 깊은 인사이트를 주는 완벽한 기술 블로그 포스트를 Markdown 형식으로 작성하세요.
    """
    final_post = editor_llm.invoke(prompt).content

    diagram_desc = "사용자가 AR 글래스로 가상 버튼을 누르면, AI 로봇이 물리적으로 전등을 켜는 과정을 보여주는 순서도"
    mermaid_code = ""
    try:
        mermaid_res = mcp_client.call_tool("mermaid_generator", {"description": diagram_desc})

        if 'error' in mermaid_res:
             mermaid_code = "graph TD; A[Mermaid 다이어그램 생성 오류] --> B[MCP 서버 응답 확인 필요];"
        else:
            result_str = mermaid_res.get('result', '{}')
            try:
                result_data = json.loads(result_str)
                mermaid_code = result_data.get('mermaid_code', "graph TD; A[Mermaid 코드 없음] --> B[응답 구조 문제];")
            except json.JSONDecodeError:
                mermaid_code = "graph TD; A[JSON 파싱 오류] --> B[응답 형식 문제];"

    except Exception as e:
        print(f" ADK Service: mermaid_generator 호출 실패: {e}")
        mermaid_code = "graph TD; A[Mermaid 생성 예외 발생] --> B[ADK 서비스 로그 확인 필요];"

    final_report_with_diagram = f"{final_post}\\n\\n## 핵심 기술 아키텍처\\n\\n```mermaid\\n{mermaid_code}\\n```"

    print("[ADK Service]  최종 편집 및 시각화 자료 생성이 완료되었습니다.")
    return final_report_with_diagram

#### 1.7 시스템의 두뇌: 통합 메모리 시스템 구현 (`memory_system.py`)

Agent가 과거의 경험으로부터 학습할 수 있도록, 3일차에 완성했던 `MemorySystem`의 완전한 코드를 재구성합니다. 이 시스템은 `Redis`를 단기 기억(대화 맥락)으로, `ChromaDB`를 장기 기억(과거 지식, 사용자 선호도)으로 사용합니다.

In [ ]:
%%writefile memory_system.py

import json
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
import redis
import chromadb
from langchain.schema import Document
from pydantic import BaseModel, Field
from typing import List
import instructor
import uuid

class ShortTermMemoryManager:
    def __init__(self, client: redis.Redis):
        self._client = client
        self._llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0)

    def _get_session_key(self, session_id: str) -> str:
        return f"session:short_term:{session_id}"

    def log_message(self, session_id: str, role: str, content: str):
        key = self._get_session_key(session_id)
        message = json.dumps({"role": role, "content": content})
        self._client.rpush(key, message)
        self._client.expire(key, 86400)

    def get_history(self, session_id: str) -> list:
        key = self._get_session_key(session_id)
        messages = self._client.lrange(key, 0, -1)
        return [json.loads(msg) for msg in messages]

    def summarize_and_get_context(self, session_id: str, max_tokens: int = 1500) -> str:
        history = self.get_history(session_id)
        if not history:
            return "(새로운 대화입니다. 이전 컨텍스트 없음)"
        history_text = "\n".join([f"{msg['role']}: {msg['content']}" for msg in history])
        if len(history_text) / 2.5 > max_tokens:
            prompt = f"다음 대화 기록의 핵심 내용을 간결하게 요약해줘: {history_text}"
            try:
                summary = self._llm.invoke(prompt).content
                return f"(이전 대화 요약): {summary}"
            except Exception:
                return f"(최근 대화): ...{history_text[-max_tokens:]}"
        else:
            return f"(이전 대화 기록):\n{history_text}"

class UserPreferences(BaseModel):
    writing_style: str = Field(description="사용자가 선호하는 글쓰기 스타일 (예: 전문적, 친근함)")
    preferred_format: str = Field(description="사용자가 선호하는 글의 구조 (예: 두괄식, Q&A 형식)")
    key_topics_of_interest: List[str] = Field(description="사용자의 주요 관심 토픽")

class LongTermMemoryManager:
    def __init__(self, client: chromadb.Client):
        self._client = client
        self._embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
        self.collection = self._client.get_or_create_collection(name="unified_agent_knowledge_base", metadata={"hnsw:space": "cosine"})

    def add_memory(self, content: str, memory_type: str, metadata: dict):
        doc_id = str(uuid.uuid4())
        metadata['memory_type'] = memory_type
        self.collection.add(documents=[content], metadatas=[metadata], ids=[doc_id])
        print(f"🧠 새로운 장기 기억 추가 (Type: {memory_type})")

    def search_memory(self, query: str, n_results: int = 3, memory_type: str = None) -> list:
        where_filter = {"memory_type": memory_type} if memory_type else {}
        results = self.collection.query(query_texts=[query], n_results=n_results, where=where_filter)
        retrieved_docs = []
        if results and results['ids'] and results['ids'][0]:
            for i in range(len(results['ids'][0])):
                doc = Document(page_content=results['documents'][0][i], metadata=results['metadatas'][0][i])
                retrieved_docs.append(doc)
        return retrieved_docs

class MemorySystem:
    def __init__(self, short_term: ShortTermMemoryManager, long_term: LongTermMemoryManager):
        self.short_term = short_term
        self.long_term = long_term
        self.preference_extractor = instructor.from_provider('google/gemini-2.5-flash-lite')

    def extract_and_store_preferences(self, session_id: str):
        history_text = "\n".join([f"{msg['role']}: {msg['content']}" for msg in self.short_term.get_history(session_id)])
        if not history_text:
            return
        prompt = f"다음 대화 기록을 분석하여 사용자의 글쓰기 선호도를 요약해줘: {history_text}"
        try:
            preferences = self.preference_extractor.chat.completions.create(messages=[{"role": "user", "content": prompt}], response_model=UserPreferences)
            self.long_term.add_memory(content=preferences.model_dump_json(indent=2), memory_type="user_preference", metadata={"session_id": session_id})
        except Exception as e:
            print(f"! 선호도 추출 오류: {e}")

    def retrieve_comprehensive_context(self, session_id: str, query: str) -> str:
        print(f"📚 쿼리 '{query[:30]}...'에 대한 종합 컨텍스트 생성 중...")
        short_term_context = self.short_term.summarize_and_get_context(session_id)
        preferences = self.long_term.search_memory("사용자 글쓰기 선호도", n_results=1, memory_type="user_preference")
        past_knowledge = self.long_term.search_memory(query, n_results=2, memory_type="final_report")

        full_context = "--- 종합 컨텍스트 ---\n"
        full_context += f"[현재 대화 맥락]\n{short_term_context}\n"
        if preferences:
            full_context += f"\n[학습된 사용자 선호도]\n{preferences[0].page_content}\n"
        if past_knowledge:
            knowledge_str = "\n".join([f"- (과거 보고서): {doc.page_content[:200]}..." for doc in past_knowledge])
            full_context += f"\n[관련 과거 지식]\n{knowledge_str}\n"
        full_context += "--- 컨텍스트 끝 ---"
        return full_context

#### 1.8 `LangGraph` 오케스트레이터 구현 (`service_langgraph_orchestrator.py`)

전체 워크플로우를 총괄하는 `LangGraph` 오케스트레이터 로직을 구현합니다. 작업의 중간 결과를 바탕으로 다음 단계를 동적으로 결정하는 조건부 라우팅 로직을 포함합니다.

In [ ]:
%%writefile service_langgraph_orchestrator.py

from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List, Optional
import operator
from langfuse.langchain import CallbackHandler
from langfuse import observe

from memory_system import MemorySystem
from service_crewai_research import run_research_crew
from service_adk_quality import run_quality_control
from mcp_client import MCPClient

class OrchestratorState(TypedDict):
    session_id: str
    user_input: str
    topic: str
    user_preferences: str
    memory_context: str
    research_report: Optional[str]
    final_report: Optional[str]
    log: Annotated[List[str], operator.add]

@observe(name="orchestrator-node-memory-retrieval")
def memory_retrieval_node(state: OrchestratorState, memory_system: MemorySystem) -> dict:
    print("--- Node: 🧠 Memory Retrieval ---")
    session_id = state['session_id']
    user_input = state['user_input']
    context = memory_system.retrieve_comprehensive_context(session_id, user_input)
    topic = user_input
    preferences = "전문적이면서도 이해하기 쉬운 톤으로 작성"
    return {"memory_context": context, "topic": topic, "user_preferences": preferences, "log": ["Memory context retrieved."]}

@observe(name="orchestrator-node-research")
def research_node(state: OrchestratorState, mcp_client: MCPClient) -> dict:
    print("--- Node: ✍️  Delegating to CrewAI Research Team ---")
    report = run_research_crew(state['topic'], state['user_preferences'], state['memory_context'], mcp_client)
    return {"research_report": report, "log": ["CrewAI research and drafting completed."]}

@observe(name="orchestrator-node-quality-control")
def quality_control_node(state: OrchestratorState, mcp_client: MCPClient) -> dict:
    print("--- Node: 🧐 Delegating to ADK Quality Team ---")
    final_report = run_quality_control(state['topic'], state['research_report'], mcp_client)
    return {"final_report": final_report, "log": ["ADK quality control and finalization completed."]}

def create_orchestrator(memory_system: MemorySystem, mcp_client: MCPClient):
    workflow = StateGraph(OrchestratorState)

    workflow.add_node("memory_retrieval", lambda state: memory_retrieval_node(state, memory_system))
    workflow.add_node("research", lambda state: research_node(state, mcp_client))
    workflow.add_node("quality_control", lambda state: quality_control_node(state, mcp_client))

    workflow.set_entry_point("memory_retrieval")
    workflow.add_edge("memory_retrieval", "research")
    workflow.add_edge("research", "quality_control")
    workflow.add_edge("quality_control", END)

    return workflow.compile()

#### 1.9 A2A 유사 게이트웨이 구현 (`a2a_gateway.py`)

마지막으로, 외부의 요청을 받아 처리하는  `FastAPI` 기반 게이트웨이의 코드를 생성합니다.  

이 게이트웨이는 `LangGraph` 오케스트레이터를 호출하고, `Langfuse`를 통해 요청에 대한 최상위 Trace를 생성하는 역할을 합니다.

In [ ]:
%%writefile a2a_gateway.py

from fastapi import FastAPI, Depends, HTTPException, Security, Request
from fastapi.security import APIKeyHeader
from contextlib import asynccontextmanager
import os
import redis
import chromadb
import traceback

from langfuse import Langfuse
from langfuse.langchain import CallbackHandler

from a2a_protocol import OrchestrationRequest, OrchestrationResponse
from mcp_client import MCPClient
from memory_system import MemorySystem, ShortTermMemoryManager, LongTermMemoryManager
from service_langgraph_orchestrator import create_orchestrator

@asynccontextmanager
async def lifespan(app: FastAPI):
    app.state.is_ready = False
    try:
        mcp_server_url = "http://localhost:8501"
        master_api_key = os.environ["MASTER_API_KEY"]

        app.state.langfuse_client = Langfuse()
        app.state.mcp_client = MCPClient(base_url=mcp_server_url, api_key=master_api_key)

        upstash_rest_url = os.environ['UPSTASH_REDIS_REST_URL']
        upstash_rest_token = os.environ['UPSTASH_REDIS_REST_TOKEN']
        full_redis_url = f"rediss://:{upstash_rest_token}@{upstash_rest_url.replace('https://', '')}"

        redis_client = redis.Redis.from_url(full_redis_url, decode_responses=True)
        chroma_client = chromadb.PersistentClient(path="final_project_db")
        app.state.memory_system = MemorySystem(ShortTermMemoryManager(redis_client), LongTermMemoryManager(chroma_client))
        app.state.orchestrator = create_orchestrator(app.state.memory_system, app.state.mcp_client)

        app.state.is_ready = True
        print("--- [A2A Gateway] LIFESPAN: All components initialized successfully. Server is ready. ---")

    except Exception as e:
        print("\n" + "="*80)
        print("! [A2A Gateway] LIFESPAN: CRITICAL STARTUP ERROR! ")
        print(traceback.format_exc())
        print("="*80 + "\n")
        app.state.orchestrator = None
        app.state.memory_system = None

    yield
    if getattr(app.state, 'langfuse_client', None):
        app.state.langfuse_client.flush()

app = FastAPI(title="A2A Gateway for Sentient Architect", version="1.0.0", lifespan=lifespan)
API_KEY_NAME = "X-API-Key"
api_key_header = APIKeyHeader(name=API_KEY_NAME, auto_error=False)

async def get_api_key(api_key: str = Security(api_key_header)):
    if api_key == os.environ.get("MASTER_API_KEY"):
        return api_key
    raise HTTPException(status_code=401, detail="Invalid API Key")

@app.get("/", tags=["Status"])
async def root(request: Request):
    if getattr(request.app.state, 'is_ready', False):
        return {"status": "ok", "message": "A2A Gateway is alive and well."}
    else:
        raise HTTPException(status_code=503, detail="Service not ready. Check startup logs.")

@app.post("/api/v1/generate-deep-research-report", response_model=OrchestrationResponse, tags=["Core Workflow"])
async def generate_report(fastapi_req: Request, data: OrchestrationRequest, api_key: str = Depends(get_api_key)):
    orchestrator = fastapi_req.app.state.orchestrator
    memory_system = fastapi_req.app.state.memory_system

    if not orchestrator or not memory_system:
        raise HTTPException(status_code=503, detail="Orchestrator or MemorySystem not initialized. Check gateway startup logs.")

    # 초기 상태에 모든 필수 키를 미리 설정
    initial_state = {
        "session_id": data.session_id,
        "user_input": data.user_input,
        "topic": "",
        "user_preferences": "",
        "memory_context": "",
        "research_report": None,
        "final_report": None,
        "log": []
    }

    try:
        print(f"[A2A Gateway] Starting orchestration for session: {data.session_id}")

        # LangGraph orchestrator 실행
        final_state = orchestrator.invoke(initial_state)

        print(f"[A2A Gateway] Orchestration completed. Final state keys: {list(final_state.keys())}")

        # 안전하게 결과 추출
        final_report_content = final_state.get('final_report', '')
        topic_content = final_state.get('topic', 'Unknown Topic')

        # 메모리에 저장 (내용이 있을 때만)
        if final_report_content and len(final_report_content.strip()) > 0:
            try:
                memory_system.long_term.add_memory(
                    content=final_report_content,
                    memory_type="final_report",
                    metadata={"topic": topic_content}
                )
                memory_system.extract_and_store_preferences(data.session_id)
                print("[A2A Gateway] Successfully stored results in long-term memory")
            except Exception as mem_error:
                print(f"[A2A Gateway] Warning: Memory storage failed: {mem_error}")

        # 성공 응답 반환
        response = OrchestrationResponse(
            session_id=data.session_id,
            final_report=final_report_content,
            status="COMPLETED"
        )

        print(f"[A2A Gateway] Successfully generated report with {len(final_report_content)} characters")
        return response

    except Exception as e:
        print(f"! [A2A Gateway] ERROR during report generation:")
        print(traceback.format_exc())

        # 에러 응답 반환
        return OrchestrationResponse(
            session_id=data.session_id,
            final_report="",
            status="FAILED",
            error_message=str(e)
        )

### 2. 시스템 실행

이제 모든 설계와 구현이 끝났습니다. 에이전트를 가동하고, 단일 요청으로 전체 에이전트가 작업을 수행하는 과정을 살펴보겠습니다.

먼저, 두 개의 FastAPI 서버를 안정적으로 실행하고 외부에서 접근할 수 있도록 해주는 핵심 유틸리티인 서버 런처를 준비합니다.

#### 2.1 런처 실행

In [ ]:
import os
import sys
import subprocess
import time
import requests
import nest_asyncio
import threading
from IPython.display import display, HTML

nest_asyncio.apply()

upstash_url = os.environ.get("UPSTASH_REDIS_REST_URL", "").replace("https://", "")
upstash_token = os.environ.get("UPSTASH_REDIS_REST_TOKEN", "")
os.environ["UPSTASH_REDIS_URL"] = f"rediss://:{upstash_token}@{upstash_url}"
print(" 외부 서비스 연결 정보 설정 완료.")

print("\n--- [Step 1] 모든 기존 프로세스를 정리합니다... ---")
subprocess.run("lsof -t -i:8501 | xargs -r kill -9", shell=True, capture_output=True)
subprocess.run("lsof -t -i:8502 | xargs -r kill -9", shell=True, capture_output=True)
print(" 기존 서버 프로세스 정리 완료.")
time.sleep(2)

print("\n--- [Step 2] MCP 및 A2A 서버를 백그라운드에서 시작합니다... ---")


def unified_logger(process, name):
    # stdout과 stderr를 통합하여 실시간으로 출력합니다.
    for line in iter(process.stdout.readline, ""):
        if line:
            print(f"[{name} LOG] {line.strip()}")
    for line in iter(process.stderr.readline, ""):
        if line:
            print(f"[{name} ERR] {line.strip()}")


current_env = os.environ.copy()

# MCP 서버 시작
mcp_process = subprocess.Popen(
    [sys.executable, "-m", "uvicorn", "mcp_server:app", "--host", "0.0.0.0", "--port", "8501", "--reload"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
    encoding="utf-8",
    env=current_env,
)
threading.Thread(target=unified_logger, args=(mcp_process, "mcp_server"), daemon=True).start()
print(" MCP 서버(8501) 시작 중...")

# A2A 게이트웨이 시작
a2a_process = subprocess.Popen(
    [sys.executable, "-m", "uvicorn", "a2a_gateway:app", "--host", "0.0.0.0", "--port", "8502", "--reload"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
    encoding="utf-8",
    env=current_env,
)
threading.Thread(target=unified_logger, args=(a2a_process, "a2a_gateway"), daemon=True).start()
print("🚀 A2A 게이트웨이(8502) 시작 중...")

print("\n--- [Step 3] 서버 URL 설정 ---")
mcp_url = "http://localhost:8501"
a2a_url = "http://localhost:8502"
os.environ["MCP_SERVER_URL"] = mcp_url
print(f" MCP 서버 URL: {mcp_url}")
print(f" A2A 게이트웨이 URL: {a2a_url}")
display(HTML(f'<b><a href="{mcp_url}/docs" target="_blank"> MCP 서버 API Docs</a></b>'))
display(HTML(f'<b><a href="{a2a_url}/docs" target="_blank"> A2A 게이트웨이 API Docs</a></b>'))

print("\n--- [Step 4] 시스템 상태 최종 확인 --- ")


def wait_for_server(url, server_name, max_retries=20, delay=2):
    print(f"    '{server_name}' 서버의 응답을 기다리는 중...")
    for i in range(max_retries):
        try:
            response = requests.get(url, timeout=5)
            if response.status_code == 200:
                print(f"    '{server_name}' 서버 준비 완료! (상태 코드: 200)")
                return True
            else:  # 503 에러 등을 포함
                print(f"   ... 대기 중 ({i+1}/{max_retries}) - 상태 코드: {response.status_code}")
        except requests.exceptions.RequestException:
            print(f"   ... 대기 중 ({i+1}/{max_retries}) - 아직 연결할 수 없음")
        time.sleep(delay)
    print(f"    '{server_name}' 서버가 시간 내에 응답하지 않았습니다.")
    return False


if wait_for_server(mcp_url, "MCP Server") and wait_for_server(a2a_url, "A2A Gateway"):
    print("\n 모든 시스템이 성공적으로 온라인 상태가 되었습니다. 최종 실행을 진행할 수 있습니다.")
else:
    print("\n! 일부 시스템 시작에 실패했습니다. 위의 로그에서 진짜 원인을 찾아주세요.")

#### 2.2 오케스트레이션 실행

아래 코드는 최종 사용자 또는 상위 애플리케이션의 역할을 수행하는 '지휘자 클라이언트'입니다. 이 클라이언트는 게이트웨이의 엔드포인트(`/api/v1/generate-deep-research-report`)에 요청을 보내는 것으로, 시스템 전체를 가동시킵니다.

실행을 시작하면, A2A 게이트웨이와 MCP 서버의 실시간 로그, 그리고 이 클라이언트 셀의 출력을 통해 전체 워크플로우가 어떻게 진행되는지, LangGraph 오케스트레이터가 CrewAI 팀과 ADK 팀에게 어떻게 작업을 위임하는지 그 과정을 관찰할 수 있습니다.

In [ ]:
import requests
import uuid
import json
from IPython.display import display, Markdown, HTML
import time

if "a2a_url" in locals() and a2a_url:
    # --- 1. 최종 실행을 위한 요청 데이터 준비 ---
    session_id = f"final-project-session-{uuid.uuid4().hex[:8]}"
    user_request = "차세대 인간-컴퓨터 상호작용(HCI): 공간 컴퓨팅(Spatial Computing)과 물리적 AI 에이전트(Embodied AI)의 융합이 가져올 산업 및 사회 변화에 대한 심층 분석 보고서를 작성해줘."
    headers = {
        "X-API-Key": os.environ["MASTER_API_KEY"],
        "Content-Type": "application/json",
    }
    payload = {"session_id": session_id, "user_input": user_request}

    print("--- 🚀 최종 오케스트레이션 클라이언트 실행 ---")
    print(f"세션 ID: {session_id}")
    print(f"요청 주제: {user_request}")
    print("A2A 게이트웨이로 요청을 전송합니다... (이 과정은 몇 분 정도 소요될 수 있습니다)")

    # --- 2. A2A 게이트웨이에 최종 보고서 생성 요청 ---
    try:
        start_time = time.time()
        response = requests.post(
            f"{a2a_url}/api/v1/generate-deep-research-report", json=payload, headers=headers, timeout=600
        )
        end_time = time.time()
        response.raise_for_status()

        result_data = response.json()

        if result_data.get("status") == "FAILED":
            print(f"\n A2A 게이트웨이에서 작업 실패 응답을 받았습니다 (소요 시간: {end_time - start_time:.2f}초).")
            print(f"   - 서버 오류 메시지: {result_data.get('error_message')}")

            if result_data.get("trace_url"):
                display(
                    HTML(
                        f'<b><a href="{result_data["trace_url"]}" target="_blank">👉 Langfuse에서 실패 원인을 추적해보세요.</a></b>'
                    )
                )
        else:
            # --- 3. 성공 시 최종 결과물 출력 ---
            print(f"\n---  작업 완료 (총 소요 시간: {end_time - start_time:.2f}초) ---")
            print("\n" + "=" * 80)
            print("                            최종 심층 분석 보고서 ")
            print("=" * 80)
            display(Markdown(result_data.get("final_report", "결과 보고서가 없습니다.")))

            # --- 4. Langfuse 추적 URL 제공 ---
            if result_data.get("trace_url"):
                trace_url = result_data["trace_url"]
                print("\n" + "=" * 80)
                print("        🕵️‍♂️ Langfuse에서 전체 실행 과정 추적하기")
                print("=" * 80)
                display(
                    HTML(
                        f'<b><a href="{trace_url}" target="_blank">👉 여기를 클릭하여 엔드투엔드 Trace를 확인하세요.</a></b>'
                    )
                )

    except requests.exceptions.HTTPError as e:
        print(f"\n HTTP 오류 발생: {e.response.status_code}")
        print(f"   - 응답 내용: {e.response.text}")
    except requests.exceptions.RequestException as e:
        print(f"\n 요청 실패: {e}")
        print("   - 서버가 정상적으로 실행 중인지, URL이 올바른지 확인해주세요.")
else:
    print(" A2A 게이트웨이 URL이 설정되지 않았습니다. 이전 셀을 먼저 성공적으로 실행해주세요.")

### 3. 시스템 평가

이 파트에서는 `Langfuse`를 활용하여 우리 시스템의 성능을 정량적으로 측정하고, 데이터 기반으로 개선할 수 있는 평가 파이프라인(Evaluation Pipeline)을 구축합니다.

#### 3.1 벤치마크 생성

먼저 Agent 시스템의 능력을 평가하기 위한 `Langfuse Dataset`을 설계합니다. 이 데이터셋은 다양한 측면의 문제와, 그에 대한 모범 답안으로 구성됩니다.

In [ ]:
evaluation_benchmark_data = [
    {
        "name": "Technical Comparison",
        "input": "Apple Vision Pro와 Meta Quest 3의 디스플레이 기술(해상도, PPD, 패널 타입)을 비교하고, 공간 컴퓨팅 경험에 미치는 영향의 차이점을 기술적으로 분석하시오.",
        "expected_output": "Apple Vision Pro는 Micro-OLED 패널을 사용하여 인치당 3,386 픽셀의 초고해상도를 달성, 약 34-40 PPD(Pixels Per Degree)를 제공하여 현실과 거의 구분이 불가능한 선명함을 제공합니다. 이는 텍스트 가독성을 극대화하고 장시간 사용 시의 피로도를 줄여줍니다. 반면 Meta Quest 3는 LCD 패널을 사용하며 약 25 PPD를 제공하여 Vision Pro 대비 선명도는 떨어지지만, 더 넓은 시야각과 비용 효율성을 확보했습니다. 이 차이로 인해 Vision Pro는 '작업 생산성'과 '미디어 소비'에, Quest 3는 '게임'과 '대중적 VR 경험'에 더 강점을 가집니다.",
    },
    {
        "name": "Creative Scenario",
        "input": "물리적 AI 에이전트(Embodied AI)가 미래의 가정 내 노인 돌봄(elderly care)을 어떻게 혁신할 수 있는지, 구체적인 기술과 함께 창의적인 시나리오를 3가지 제시하시오.",
        "expected_output": "1. 동반자 및 건강 모니터링 에이전트: SLAM 기술로 집안을 자유롭게 이동하며, 컴퓨터 비전으로 노인의 얼굴 표정과 걸음걸이를 분석하여 건강 이상 징후를 조기에 발견하고, 대화형 AI로 정서적 지지를 제공합니다. 2. 맞춤형 재활 보조 에이전트: 3D 센서로 노인의 움직임을 정밀하게 추적하며, 재활 운동의 정확한 자세를 코칭하고 게임화된 콘텐츠로 재활의 동기를 부여합니다. 3. 응급 상황 대응 에이전트: 낙상 감지 센서와 생체 신호 모니터링을 통해 응급 상황 발생 시 즉시 119에 자동으로 신고하고, 가족에게 영상 통화를 연결하여 상황을 전달하는 역할을 수행합니다.",
    },
    {
        "name": "Ethical Challenge Analysis",
        "input": "'상시 작동하는(always-on)' 공간 컴퓨팅 기기가 대중화될 때 발생할 수 있는 가장 심각한 프라이버시 문제 3가지를 식별하고, 이에 대한 잠재적인 기술적/정책적 해결책을 논하시오.",
        "expected_output": "1. 공간 데이터 유출: 기기가 수집하는 3D 공간 정보(집 내부 구조, 방문객 등)가 해킹되거나 오용될 수 있습니다. (해결책: 온디바이스 AI 처리 강화, 데이터 최소화 원칙 적용, 연합 학습 도입). 2. 생체 정보의 상업적 이용: 시선 추적, 표정, 뇌파(미래) 데이터가 사용자의 동의 없이 광고나 마케팅에 활용될 수 있습니다. (해결책: 강력한 개인정보보호법(GDPR 등) 확대 적용, 데이터 사용처에 대한 투명한 공개 의무화). 3. 사회적 감시 및 통제: 공공장소에서 타인의 행동과 대화가 무분별하게 기록되어 사회적 상호작용을 위축시킬 수 있습니다. (해결책: 녹화 시 명확한 시각적 표시(LED 등) 의무화, 특정 장소(화장실 등)에서의 자동 비활성화 기능 탑재, 관련 법규 제정.",
    },
    {
        "name": "Business Strategy",
        "input": "삼성이 2026년에 소비자용 AR 글래스 시장에 진출한다고 가정할 때, Apple Vision Pro와 차별화하기 위한 가장 효과적인 시장 진입 전략은 무엇일까? 제품 포지셔닝과 핵심 기능 관점에서 제안하시오.",
        "expected_output": "삼성은 Apple의 '최고급 공간 컴퓨터' 포지셔닝을 피하고, '일상과 매끄럽게 통합되는 AI 비서'로 포지셔닝해야 합니다. 이를 위해 1) 경량화 및 디자인: 하루 종일 착용해도 부담 없는 일반 안경 형태의 디자인을 최우선으로 합니다. 2) 갤럭시 생태계 연동: 스마트폰, 워치, 가전제품과 유기적으로 연동하여, AR 글래스가 모든 디바이스를 제어하는 '컨트롤 타워' 역할을 수행하게 합니다. 3) 가격 경쟁력: 100만원대 초반의 공격적인 가격 정책으로 시장 진입 장벽을 낮춰 대중화를 선도하는 전략이 효과적일 것입니다. 핵심 기능은 고사양 게임이 아닌, 실시간 통역, 길안내, AI 비서(온디바이스 '가우스')와의 상호작용에 집중해야 합니다.",
    },
]

print(f" 총 {len(evaluation_benchmark_data)}개의 평가 문항이 설계되었습니다.")

#### 3.2 벤치마크 생성: Langfuse 데이터셋 업로드

설계한 평가 문항들을 `Langfuse` 서버에 데이터셋으로 업로드합니다.

In [ ]:
from langfuse import Langfuse

try:
    langfuse = Langfuse()
    langfuse.auth_check()
    print("Langfuse 클라이언트가 성공적으로 연결되었습니다.")

    DATASET_NAME = "Sentient_Architect_Benchmark_v1"

    # 데이터셋이 이미 있는지 확인
    try:
        dataset = langfuse.get_dataset(name=DATASET_NAME)
        print(f"데이터셋 '{DATASET_NAME}'은(는) 이미 존재합니다. 기존 데이터셋을 사용합니다.")
    except Exception:
        # 존재하지 않으면 새로 생성
        langfuse.create_dataset(name=DATASET_NAME)
        print(f"새로운 데이터셋 '{DATASET_NAME}'을 생성했습니다.")

    # 데이터셋 아이템 업로드
    print("\n--- 데이터셋 아이템 업로드 시작 ---")
    for item in evaluation_benchmark_data:
        # 동일한 input을 가진 아이템이 이미 있는지 확인하여 중복 방지
        existing_items = langfuse.get_dataset(DATASET_NAME).items
        is_duplicate = any(i.input == item["input"] for i in existing_items)

        if not is_duplicate:
            langfuse.create_dataset_item(
                dataset_name=DATASET_NAME,
                input=item["input"],
                expected_output=item["expected_output"],
                metadata={"name": item["name"]},
            )
            print(f"  - Item '{item['name']}' 업로드 성공")
        else:
            print(f"  - Item '{item['name']}'은(는) 이미 존재하여 건너뜁니다.")

    print("\n모든 평가 문항이 Langfuse 데이터셋에 성공적으로 준비되었습니다.")

except Exception as e:
    print(f" Langfuse 연결 또는 데이터셋 생성 중 오류가 발생했습니다: {e}")
    print("   - Langfuse API 키가 올바르게 설정되었는지 확인해주세요.")

#### 3.3 자동 채점관 구현: LLM-as-a-Judge

우리 시스템이 생성한 복잡한 보고서를 일관된 기준으로 평가하기 위해, 'AI 채점관' 역할을 할 `LLM-as-a-Judge`를 구현합니다. 이 채점관은 세 가지 구체적인 평가 기준(기술적 깊이, 논리적 명확성, 독창성)에 대해 각각 점수를 매기고, 그 근거를 서술하도록 설계하여 단순한 점수 이상의 깊이 있는 피드백을 제공합니다.

In [ ]:
import instructor
from pydantic import BaseModel, Field
from typing import List
from langfuse import Langfuse

# 평가자 LLM 클라이언트 초기화
try:
    evaluator_llm_client = instructor.from_provider("google/gemini-2.5-flash", api_key=os.environ.get("GOOGLE_API_KEY"))
    print(" LLM-as-a-Judge를 위한 'gemini-2.5-flash' 평가자 LLM이 준비되었습니다.")
except Exception as e:
    print(f" 평가자 LLM 초기화 실패: {e}")
    evaluator_llm_client = None


# 평가 결과를 담을 Pydantic 모델
class SubScore(BaseModel):
    criteria: str = Field(description="평가 기준")
    score: float = Field(description="해당 기준에 대한 점수 (0.0 ~ 1.0)")
    reasoning: str = Field(description="점수를 매긴 구체적인 이유")


class ComprehensiveEvaluation(BaseModel):
    """AI Agent의 답변에 대한 종합적인 평가 결과"""

    overall_score: float = Field(description="모든 기준을 종합한 최종 평균 점수 (0.0 ~ 1.0)")
    scores: List[SubScore] = Field(description="세부 평가 기준별 점수 및 근거")
    overall_comment: str = Field(description="답변에 대한 총평 및 개선 제안")


# LLM-as-a-Judge 함수
def evaluate_report_quality(
    input_question: str, generated_report: str, expected_answer: str
) -> ComprehensiveEvaluation:
    """LLM을 사용하여 생성된 보고서의 품질을 종합적으로 평가합니다."""
    if not evaluator_llm_client:
        raise ConnectionError("평가자 LLM 클라이언트가 초기화되지 않았습니다.")

    prompt = f"""당신은 세계 최고 수준의 기술 분석 보고서 평가 위원입니다.
    다음 [사용자 질문], [AI가 생성한 보고서], 그리고 [모범 답안 예시]를 바탕으로, 보고서의 품질을 세 가지 기준에 따라 엄격하고 객관적으로 평가해주세요.

    [평가 기준]
    1.  Technical_Depth (기술적 깊이): 기술적 개념을 정확하고 깊이 있게 다루었는가? 핵심적인 기술 요소를 놓치지 않았는가?
    2.  Logical_Clarity (논리적 명확성): 보고서의 구조가 논리적이며, 주장이 명확하고, 근거가 타당한가? 독자가 이해하기 쉬운가?
    3.  Insightfulness (독창성 및 통찰력): 단순히 정보를 나열하는 것을 넘어, 독창적인 분석이나 깊이 있는 통찰력을 제공하는가? 모범 답안의 핵심을 잘 파악했는가?

    --- START OF DATA ---
    [사용자 질문]: {input_question}

    [AI가 생성한 보고서]:\n{generated_report}

    [모범 답안 예시]:\n{expected_answer}
    --- END OF DATA ---

    위 내용을 바탕으로 ComprehensiveEvaluation JSON 객체를 생성해주세요.
    """

    evaluation_result = evaluator_llm_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt.format(
                    input_question=input_question, generated_report=generated_report, expected_answer=expected_answer
                ),
            }
        ],
        response_model=ComprehensiveEvaluation,
    )
    return evaluation_result

#### 3.4 평가 파이프라인 실행

이제 모든 준비가 끝났습니다. 아래 코드는 '표준 시험지(Dataset)'의 각 문항을 우리 시스템에 하나씩 풀게 하고, 그 결과물을 'AI 채점관(LLM-as-a-Judge)'이 채점한 뒤, 모든 과정을 `Langfuse`에 자동으로 기록하는 전체 평가 파이프라인입니다.

In [ ]:
import instructor
from pydantic import BaseModel, Field
from typing import List
import time
import requests
from tqdm.notebook import tqdm
import uuid
from langfuse import Langfuse
import os

# 평가자 LLM 클라이언트 초기화
try:
    client = instructor.from_provider("google/gemini-2.5-flash", api_key=os.environ.get("GOOGLE_API_KEY"))
    print(" LLM-as-a-Judge 평가자 준비 완료")
except Exception as e:
    print(f" 평가자 LLM 초기화 실패: {e}")
    client = None


# 평가 결과 모델
class Score(BaseModel):
    technical: float = Field(description="Technical accuracy score 0.0-1.0")
    clarity: float = Field(description="Logical clarity score 0.0-1.0")
    insight: float = Field(description="Insightfulness score 0.0-1.0")
    comment: str = Field(description="Brief evaluation comment")


def evaluate_report_quality(question: str, report: str, expected: str) -> Score:
    """LLM을 사용하여 생성된 보고서의 품질을 종합적으로 평가합니다."""
    if not client:
        return Score(technical=0.5, clarity=0.5, insight=0.5, comment="Client not available")

    prompt = f"""Evaluate this technical report:

Question: {question}

Generated Report: {report}

Reference Answer: {expected}

Rate on 3 criteria (0.0-1.0 scale):
- Technical accuracy and depth
- Logical clarity and structure
- Insightfulness and creativity

Provide scores and brief comment."""

    try:
        evaluation = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}], response_model=Score, max_retries=3
        )
        return evaluation

    except Exception as e:
        return Score(technical=0.5, clarity=0.5, insight=0.5, comment=f"Evaluation error: {type(e).__name__}")


def run_evaluation_pipeline():
    """완전 자동화된 평가 파이프라인"""
    if "a2a_url" not in globals() or not globals()["a2a_url"]:
        print(" A2A 게이트웨이가 실행되지 않았습니다.")
        return

    try:
        langfuse = Langfuse()
        dataset = langfuse.get_dataset(name="Sentient_Architect_Benchmark_v1")
        print(f" 데이터셋 로드 완료: {len(dataset.items)}개 항목")
    except Exception as e:
        print(f" 데이터셋 로드 실패: {e}")
        return

    headers = {
        "X-API-Key": os.environ["MASTER_API_KEY"],
        "Content-Type": "application/json",
    }

    print("--- 🚀 평가 시작 ---")

    results = []

    for idx, item in enumerate(tqdm(dataset.items, desc="평가 진행")):
        session_id = f"eval-{uuid.uuid4().hex[:8]}"
        print(f"\n[{idx+1}/{len(dataset.items)}] {item.metadata['name']}")

        with item.run(run_name=f"eval-{item.metadata['name']}-{session_id}") as run_manager:
            try:
                # 1. 보고서 생성
                response = requests.post(
                    f"{a2a_url}/api/v1/generate-deep-research-report",
                    json={"session_id": session_id, "user_input": item.input},
                    headers=headers,
                    timeout=600,
                )

                if response.status_code != 200:
                    raise Exception(f"API 오류: {response.status_code} - {response.text}")

                report = response.json().get("final_report", "")
                if not report:
                    raise Exception("빈 보고서 수신")

                print(f"   보고서 생성 완료 ({len(report)}자)")
                run_manager.update(output=report)

                # 2. 평가 수행
                score = evaluate_report_quality(question=item.input, report=report, expected=item.expected_output)

                # 3. 점수 기록
                overall = (score.technical + score.clarity + score.insight) / 3

                run_manager.score(name="Overall_Score", value=overall, comment=score.comment)
                run_manager.score(name="Technical_Accuracy", value=score.technical)
                run_manager.score(name="Logical_Clarity", value=score.clarity)
                run_manager.score(name="Insightfulness", value=score.insight)

                results.append(
                    {
                        "name": item.metadata["name"],
                        "overall": overall,
                        "technical": score.technical,
                        "clarity": score.clarity,
                        "insight": score.insight,
                    }
                )

                print(
                    f"   평가 완료: 전체 {overall:.2f} (기술{score.technical:.2f}/논리{score.clarity:.2f}/통찰{score.insight:.2f})"
                )

            except Exception as e:
                print(f"   처리 오류: {e}")
                run_manager.update(output={"error": str(e)}, level="ERROR")
                run_manager.score(name="Overall_Score", value=0.0, comment=f"오류: {e}")

                results.append(
                    {"name": item.metadata["name"], "overall": 0.0, "technical": 0.0, "clarity": 0.0, "insight": 0.0}
                )

        time.sleep(2)

    langfuse.flush()

    # 결과 요약
    print("\n--- 📊 최종 평가 결과 ---")
    total_overall = 0
    for result in results:
        print(
            f"{result['name']:25} | 전체: {result['overall']:.2f} | 기술: {result['technical']:.2f} | 논리: {result['clarity']:.2f} | 통찰: {result['insight']:.2f}"
        )
        total_overall += result["overall"]

    avg_score = total_overall / len(results) if results else 0
    print(f"\n평균 점수: {avg_score:.2f}")
    print("---  평가 완료! Langfuse에서 상세 결과를 확인하세요. ---")


# 실행
run_evaluation_pipeline()

### Part 4: 심층 분석 및 개선


#### Langfuse 대시보드 탐험 과제 (Action Item)

1.  Langfuse Cloud ([https://cloud.langfuse.com/](https://cloud.langfuse.com/)) 로 이동하여 여러분의 프로젝트에 접속하세요.

2.  Dataset 상세 분석 페이지로 이동:
    - 왼쪽 메뉴에서 'Datasets'를 클릭하고, `Sentient_Architect_Benchmark_v1` 데이터셋을 선택합니다.
    - 'Runs' 탭을 클릭하여 Part 3에서 실행한 평가 결과를 확인합니다.

3.  종합 성능 분석:
    - 평균 점수 확인: 페이지 상단에 표시된 'Average Scores'를 확인하세요. 'Overall'의 평균 점수는 몇 점인가요? 'Technical', 'Logical', 'Insightfulness' 중 가장 점수가 낮게 나온 항목은 무엇인가요? 
    - 실행 시간 및 비용 분석: 각 실행(Run)별 'Latency' 와 'Total Cost' 를 비교해보세요. 특정 유형의 질문(예: 'Ethical Challenge')을 처리하는 데 유독 많은 시간이나 비용이 소요되지는 않았나요?

4.  실패 사례(Worst-Case) 심층 분석:
    - 'Overall' 점수가 가장 낮게 나온 실행(Run)을 찾으세요.
    - 해당 실행의 Trace 아이콘을 클릭하여 엔드투엔드 Trace 상세 페이지로 이동합니다.
    - 오케스트레이션 흐름 검토: `LangGraph` 오케스트레이터가 각 전문가 팀을 어떻게 호출했는지 확인합니다.
    - CrewAI 내부 동작 분석: `crewai-research-team-execution` Span을 확장하여, 리서치 팀 내부의 어떤 Agent가 어떤 Tool을 호출했고, 어떤 결과를 얻었는지 추적합니다. (예: `Tavily` 검색 결과가 부실하지는 않았는가?)
    - ADK 편집 과정 검토: `ADK quality control` Span을 확장하여, 최종 편집 단계에서 어떤 프롬프트가 사용되었고, 초안 대비 어떤 부분이 크게 수정되었는지 확인합니다.
    - 점수 근거 확인: Trace의 'Scores' 탭에서 'AI 채점관'이 왜 낮은 점수를 주었는지에 대한 'Comment'를 꼼꼼히 읽어보고, 실패의 근본적인 원인을 파악합니다.

